In [28]:
import pandas as pd
import numpy as np

In [33]:
file1 = pd.read_csv('/Users/avapapetti/Desktop/CNV_Files/SL88823_20180802.cnv.csv', sep = "\t")
file2 = pd.read_csv('/Users/avapapetti/Desktop/CNV_Files/SL88824_20180802.cnv.csv', sep = "\t")

file1['Sample#'] = 1
file2['Sample#'] = 2

In [34]:
min_cnv_length = 1000
p_value_threshold = .90

"""Remove CNVs from both samples that fail to meet intial criteria"""

file1 = file1.drop(file1[file1.Chrom == 'chrM'].index)
file1 = file1[(np.abs(file1.Start - file1.Stop) >= min_cnv_length) & (file1.P_Value >= p_value_threshold)]

file2 = file2.drop(file2[file2.Chrom == 'chrM'].index)
file2 = file2[(np.abs(file2.Start - file2.Stop) >= min_cnv_length) & (file2.P_Value >= p_value_threshold)]

In [35]:
def common_cnvs_finder():
    
    """Find CNVs common to both samples that meet qualifying criteria and create output file of results"""
    
    common_cnvs = pd.DataFrame(columns = file1.columns)
    file2_byChrom = file2.groupby('Chrom')

    for i in range(len(file1)):
        file2_cnvs = file2_byChrom.get_group(file1.iloc[i].Chrom)
        file2_cnvs = file2_cnvs.loc[file2_cnvs.Type.isin(file1.iloc[[i]].Type)]
        file2_cnvs = file2_cnvs.loc[(np.abs(file2_cnvs.Start.subtract(file1.iloc[i].Start)) <= 25) &
        (np.abs(file2_cnvs.Stop.subtract(file1.iloc[i].Stop)) <= 25)]

        if(len(file2_cnvs) > 0):
            common_cnvs = pd.concat([common_cnvs, file1.iloc[[i]], file2_cnvs]).reset_index(drop = True)
    
    print(common_cnvs.info())
    common_cnvs_output = common_cnvs.to_csv()

In [36]:
common_cnvs_finder()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 6 columns):
Chrom      58 non-null object
Start      58 non-null object
Stop       58 non-null object
Type       58 non-null object
P_Value    58 non-null float64
Sample#    58 non-null object
dtypes: float64(1), object(5)
memory usage: 2.8+ KB
None
